# InitPlay基础知识

## 一、几个主要的状态值



|状态|默认值|作用|
|:---:|:---:|:---:| 
|_playing|false|是否处于播放状态|
|_playIsinitialized|false|是否对播放参数进行了设置|
|_buildInAecEnabled|false|是否启用了硬件AEC功能|
|_recIsInitiaized|false|是否对录录制参数进行了设备|
|_recording|false|是否处理录制状态 |
|_speakerIsInitialized|false|是否对扬声器进行了初始化|
|_microphoneIsIntialized|false|是否对microphone进行了初始化|


## 二、音频的数据结构

```cpp
/* general extended waveform format structure
   Use this for all NON PCM formats
   (information common to all formats)
*/
#ifndef _WAVEFORMATEX_
#define _WAVEFORMATEX_

typedef struct tWAVEFORMATEX
{
    WORD    wFormatTag;        /* 设置声音格式类型 PCM  format type  */
    WORD    nChannels;         /* 设置通道数 number of channels (i.e. mono, stereo...)   */
    DWORD   nSamplesPerSec;    /* 设置采样率  sample rate */
    DWORD   nAvgBytesPerSec;   /* 设置每秒平均字节数 for buffer estimation */
    WORD    nBlockAlign;       /* 设置对齐字节数 block size of data */
    WORD    wBitsPerSample;    /*设置每个采样的大小(位深) Number of bits per sample of mono data */
    WORD    cbSize;            /* 额外消息大小 The count in bytes of the size of
                                    extra information (after cbSize) */

} WAVEFORMATEX;
typedef WAVEFORMATEX       *PWAVEFORMATEX;
typedef WAVEFORMATEX NEAR *NPWAVEFORMATEX;
typedef WAVEFORMATEX FAR  *LPWAVEFORMATEX;
#endif /* _WAVEFORMATEX_ */

#ifdef GUID_DEFINED
```

## 三、 音频接口 IAudioClient 接口


1. 该接口属于WASAPI
2. 该接口能够再音频应用程序和音频引擎之间采集音频流
3. 也能再应用程序和设备的硬件缓冲区之间创建音频流

<font color='red'>
音频设备客户端分为两种分别：
    
1. 采集客户端
2. 扬声器客户端
</font>


```cpp

 MIDL_INTERFACE("1CB9AD4C-DBFA-4c32-B178-C2F568A703B2")
    IAudioClient : public IUnknown
    {
    public:
         ////////////////////////////////////////////////////
          //        TODO@chensong 2022-07-24 初始化音频流
          ///////////////////////////////////////////////////
          /**
          * @param AUDCLNT_SHAREMODE : 共享模式/独占模式
          * @param DWORD             : 控制流创建的flag
          * @param REFERENCE_TIME    : 请求缓冲区的大小
          * @param REFERENCE_TIME    : 周期时间
          * @param WAVEFORMATEX      : 数据格式
          * @param LPCGUID           : Audio Session 的 GUID       
          * @return
          */
          
          /////////////////////////////////////////////////////////////////
          // StreamFlags 参数类型解释：
          // AUDCLNT_STREAMFLAGS_LOOPBACK		: 
          // AUDCLNT_STREAMFLAGS_EVENTCALLBACK	:
          // ...
        virtual HRESULT STDMETHODCALLTYPE Initialize( 
            /* [annotation][in] */ 
            _In_  AUDCLNT_SHAREMODE ShareMode,
            /* [annotation][in] */ 
            _In_  DWORD StreamFlags,
            /* [annotation][in] */ 
            _In_  REFERENCE_TIME hnsBufferDuration,
            /* [annotation][in] */ 
            _In_  REFERENCE_TIME hnsPeriodicity,
            /* [annotation][in] */ 
            _In_  const WAVEFORMATEX *pFormat,
            /* [annotation][in] */ 
            _In_opt_  LPCGUID AudioSessionGuid) = 0;
        
        virtual HRESULT STDMETHODCALLTYPE GetBufferSize( 
            /* [annotation][out] */ 
            _Out_  UINT32 *pNumBufferFrames) = 0;
        
        virtual HRESULT STDMETHODCALLTYPE GetStreamLatency( 
            /* [annotation][out] */ 
            _Out_  REFERENCE_TIME *phnsLatency) = 0;
        
        virtual HRESULT STDMETHODCALLTYPE GetCurrentPadding( 
            /* [annotation][out] */ 
            _Out_  UINT32 *pNumPaddingFrames) = 0;
        
        virtual HRESULT STDMETHODCALLTYPE IsFormatSupported( 
            /* [annotation][in] */ 
            _In_  AUDCLNT_SHAREMODE ShareMode,
            /* [annotation][in] */ 
            _In_  const WAVEFORMATEX *pFormat,
            /* [unique][annotation][out] */ 
            _Out_opt_  WAVEFORMATEX **ppClosestMatch) = 0;
        
        virtual HRESULT STDMETHODCALLTYPE GetMixFormat( 
            /* [annotation][out] */ 
            _Out_  WAVEFORMATEX **ppDeviceFormat) = 0;
        
        virtual HRESULT STDMETHODCALLTYPE GetDevicePeriod( 
            /* [annotation][out] */ 
            _Out_opt_  REFERENCE_TIME *phnsDefaultDevicePeriod,
            /* [annotation][out] */ 
            _Out_opt_  REFERENCE_TIME *phnsMinimumDevicePeriod) = 0;
        
        virtual HRESULT STDMETHODCALLTYPE Start( void) = 0;
        
        virtual HRESULT STDMETHODCALLTYPE Stop( void) = 0;
        
        virtual HRESULT STDMETHODCALLTYPE Reset( void) = 0;
        
        virtual HRESULT STDMETHODCALLTYPE SetEventHandle( 
            /* [in] */ HANDLE eventHandle) = 0;
        
        /////////////////////////////////////////////////////////
          //   TODO@chensong 2022-07-24 GetService 方法解释
          ////////////////////////////////////////////////////////
          /** 
          * 
          *   @param rrid    : 接口ID
          *   @param ppv     : 输出的接口对象
          *   return STDMETHODCALLTYPE 
          */
        virtual HRESULT STDMETHODCALLTYPE GetService( 
            /* [annotation][in] */ 
            _In_  REFIID riid,
            /* [annotation][iid_is][out] */ 
            _Out_  void **ppv) = 0;
        
    };
    
```


音频流的访问方式

![](./img/audio_client_connect_server.jpg)

### 1、IAuidoClient的几个重要方法

1. Initialize   （模式[共享，独享] ...）
2. GetBufferSize
3. SetEventHandle
4. GetService


#### ①、初始化音频流


[WebRTC源码中音频流初始化在 modules/audio_device/win/audio_device_core_win.cc文件中InitPlayout方法中调用的音频流初始化方法Initialize](https://github.com/chensongpoixs/cwebrtc/blob/chensong/modules/audio_device/win/audio_device_core_win.cc)

```cpp
...

////////////////////////////////////////////////////
  //        TODO@chensong 2022-07-24 初始化音频流
  ///////////////////////////////////////////////////
  /**
  * @param AUDCLNT_SHAREMODE : 共享模式/独占模式
  * @param DWORD             : 控制流创建的flag
  * @param REFERENCE_TIME    : 请求缓冲区的大小 [是与时间有关系的哈]
  * @param REFERENCE_TIME    : 周期时间 [10ms采集一次、 20ms采集一次]
  * @param WAVEFORMATEX      : 数据格式
  * @param LPCGUID           : Audio Session 的 GUID       
  * @return
  */

 /////////////////////////////////////////////////////////////////
  // StreamFlags 参数类型解释：
  // AUDCLNT_STREAMFLAGS_LOOPBACK		: 
  // AUDCLNT_STREAMFLAGS_EVENTCALLBACK	:
  // ...
  hr = _ptrClientOut->Initialize(
      AUDCLNT_SHAREMODE_SHARED,  // share Audio Engine with other applications
      AUDCLNT_STREAMFLAGS_EVENTCALLBACK,  // processing of the audio buffer by
                                          // the client will be event driven
      hnsBufferDuration,  // requested buffer capacity as a time value (in
                          // 100-nanosecond units)
      0,                  // periodicity
      &Wfx,               // selected wave format
      NULL);              // session GUID

  if (FAILED(hr)) {
    RTC_LOG(LS_ERROR) << "IAudioClient::Initialize() failed:";
  }
...

```

#### ②、GetService方法

1. 它是IAudioClient中的方法
2. 用于获取IAudioRenderClient接口
3. 用于下写数据到渲染终端的buffer

[WebRTC源码中音频流初始化在 modules/audio_device/win/audio_device_core_win.cc文件中InitPlayout方法中调用的音频流初始化方法GetService](https://github.com/chensongpoixs/cwebrtc/blob/chensong/modules/audio_device/win/audio_device_core_win.cc)

```cpp

...
    
/////////////////////////////////////////////////////////
  //   TODO@chensong 2022-07-24 GetService 方法解释
  ////////////////////////////////////////////////////////
  /** 
  * 
  *   @param rrid    : 接口ID
  *   @param ppv     : 输出的接口对象
  *   return STDMETHODCALLTYPE 
  */
  // Get an IAudioRenderClient interface.
  SAFE_RELEASE(_ptrRenderClient);
  hr = _ptrClientOut->GetService(__uuidof(IAudioRenderClient),
                                 (void**)&_ptrRenderClient);
  EXIT_ON_ERROR(hr);

...

```







